In [3]:
import os
import geopandas as gpd
from pathlib import Path
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Configuração da conexão com o Banco de Dados | Database connection setup
db_connection_str = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}@{os.getenv('DB_HOST')}:5432/{os.getenv('DB_NAME')}"
engine = create_engine(db_connection_str)

# Caminho do arquivo Shapefile | Shapefile path
shape_file = os.getenv("SIGEF_PATH")

In [5]:
# Leitura do arquivo geoespacial | Reading geospatial file
gdf = gpd.read_file(shape_file)

# Verificação inicial da área (CRS original) | Initial area check (Original CRS)
print(gdf.area.head())

# Reprojeção para sistema métrico (SIRGAS 2000 / UTM zone 22S) 
# Reprojection to metric system (SIRGAS 2000 / UTM zone 22S)
gdf_meters = gdf.to_crs(31982)

# Verificação da área em metros quadrados | Area check in square meters
print(gdf_meters.area.head())

# Persistência dos dados no PostGIS | Data persistence to PostGIS
try:
    gdf_meters.to_postgis("Sigef_Brasil_MT", engine, if_exists='fail')
except ValueError:
    print("Aviso: A tabela já existe no PostGIS. Nenhuma alteração foi feita. / Warning: Table already exists in PostGIS. No changes made.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

DataSourceError: None: No such file or directory